In [ ]:
import numpy as np
import napari
import pandas as pd
from tqdm import tqdm
import zarr
import mFISHwarp.morphology
from skimage.filters import threshold_otsu
import pickle

In [ ]:
# image path
sources = ['/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/fused/fused.n5',
          '/mnt/ampa02_data01/tmurakami/240425_whole_4color_2nd_M037-3pb/fused/fused.n5',
          '/mnt/ampa02_data01/tmurakami/240611_fos_pvalb_slc17a7/fused/fused.n5']

voxel_size = (2.0,1.3,1.3)
reference_chan = 3
upper_percentile = 99
lower_percentile = 30
napari_view = True


if napari_view:
    # launch napari to manually explore the normalization
    viewer = napari.Viewer()
    
lookup = {}
for n5_path in tqdm(sources):
    # create Zarr file object
    img_zarr = zarr.open(store=zarr.N5Store(n5_path), mode='r')
    n5_setups = list(img_zarr.keys())

    # make mask that covers only tissue to exclude the area outside of the tissue.
    img_down_ref = img_zarr[n5_setups[reference_chan]]['timepoint0']['s4'][:]
    global_thresh = threshold_otsu(img_down_ref)
    img_mask = mFISHwarp.morphology.mask_maker(img_down_ref,global_thresh)
    sub_lookup = {}

    # calculate the lower and upper values using percentile.
    for i, setup in enumerate(n5_setups):
        img_down = img_zarr[setup]['timepoint0']['s4'][:]
        # get values at the certain percentile within the mask
        lower = np.percentile(img_down[np.where(img_mask)],lower_percentile)
        upper = np.percentile(img_down[np.where(img_mask)],upper_percentile)
        sub_lookup[i] = {'lower':lower,'upper':upper}
        if napari_view:
            viewer.add_image((img_down.astype(float)-lower)/(upper-lower), contrast_limits=[0,3],blending='additive',name=setup)


    lookup[n5_path] = sub_lookup

In [ ]:
# save the lookup table as a pickle.
file = open('/mnt/ampa02_data01/tmurakami/model_training/norm_values.pkl', 'wb')
pickle.dump(lookup, file)
file.close()